In [1]:
import pymysql, os, pandas, alminer, glob
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord

In [2]:
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

In [3]:
from skaha.session import Session
import nest_asyncio
nest_asyncio.apply()

# Read in sample to be reduced

The sample to be reduced in this notebook is assembled into a simple text file in salvage-sample-selection.ipynb.

In [9]:
fpath = '/arc/projects/salvage/ALMA_reduction/samples/'
file  =  'almaquest_sample.txt'

objID_sample, year_sample, name_sample, muid_sample, guid_sample, auid_sample, proj_sample = np.loadtxt(fpath+file, unpack = True, dtype = str, usecols = [0,11,12,13,14,15,16])
z_sample, mass_sample, rpetro_sample, ra_sample, dec_sample, res_sample, mrs_sample, AL_sample, AC_sample, TP_sample = np.loadtxt(fpath+file, unpack = True, dtype = float, usecols = [1,2,3,4,5,6,7,8,9,10])

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


# ALMiner Download and Unzip with headless jobs

Here we iterate over the **ALMA Source Name**s that were matched to SDSS targets within 10" **and** cover the appropriate frequency range given the matched SDSS target's spectroscopic redshift. For each ALMA Source Name we download **ALL** observations with that specific source name, regardless of RA/Dec position. Bear in mind that for some low-z ALMA sources, there may be observations that get downloaded that have an RA and Dec that better match a higher redshift SDSS target. *That* target is not connected to this observation.

The 10" and redshift coverage check is performed in the previous stage (salvage-sample-selection.ipynb). All names loaded from the file in the previous cell should be vetted for coverage). 

**IF** there are multiple unique ALMA Source Names in the ALMA science archive that match to an SDSS target and have an appropriate frequency range, that SDSS target will be downloaded twice, but included in the same directory (indexed by its SDSS objID). Later when making the PHANGS-ALMA pipeline Measurement Set Key, these individual observations will be under the **SAME target name** and therefore should be imaged together.

**KNOWN ISSUE**: SDSS Objects with multiple ALMA observations (as defined by having different ALMA Source Names) will have multiple jobs running at once. If one starts/finishes earlier than the other, it will delete **all** the zip files in the directory. Possible solution would be to download the tar files to temporary directories, indexed by their unique ALMA Source name, then unzip them into the shared SDSS ObjID directory. **CURRENTLY TESTING THIS SOLUTION**

In [254]:
# select appropriate resources

image = "images.canfar.net/skaha/astroml:latest"
cmd = '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh'
ram=4
cores=2

session_ids = []

rerun_targets = ['587731513694355564']

# loop over galaxies and launch jobs
for i in range(len(objID_sample)):

    ID = objID_sample[i]
    #RA = ra_sample[i]
    #DEC = dec_sample[i]
    NAME = name_sample[i]
    MUID = muid_sample[i]
    #PROJ = proj_sample[i]

    UID =  MUID.split('/')[2] + '_' + MUID.split('/')[3] + '_' + MUID.split('/')[4]

    # running only on problem galaxies
    #if ID not in rerun_targets:
    #    continue

    # I think this block can be removed now...
    #TAR1 = PROJ + '_uid___' + MUID.split('/')[2] + '_' + MUID.split('/')[3] + '_' + MUID.split('/')[4] + '_001_of_001.tar'
    #TAR2 = PROJ + '_uid___' + MUID.split('/')[2] + '_' + MUID.split('/')[3] + '_' + MUID.split('/')[4] + '_auxiliary.tar'
    #ASDM = PROJ + '_uid___' + AUID.split('/')[2] + '_' + AUID.split('/')[3] + '_' + AUID.split('/')[4] + '.asdm.sdm.tar'
    #ASDM = PROJ + '_uid___' + AUID.split('/')[2] + '_' + AUID.split('/')[3] + '_*.asdm.sdm.tar
    
    # search all directories in the ALMA project folder for the relevant data
    for root, dirs, files in os.walk(f"/arc/projects/salvage/ALMA_data/{ID}/{PROJ}/"):
        if "member.uid___"+UID in dirs:
            PATH = os.path.join(root, "member.uid___"+UID)

    # if the data has already been calibrated, choose if it will be skipped
    skip_completed = True
    ms_filepath = PATH + '/calibrated/'
    ms_files = glob.glob(ms_filepath + '*.ms*')
    if (len(ms_files)>0) & skip_completed & (YEAR>2015):
        continue

    # launch headless session on the science platform
    session = Session()
    session_id = session.create(
        name  = ID,
        image = image,
        cores = cores,
        ram   = ram,
        kind  = "headless",
        cmd   = cmd,
        args  = f'{ID} {NAME} {PATH}'
    )

    print("Sesion ID: {}".format(session_id[0]))
    session_ids.append(session_id[0])


2024-03-26 06:40:04,049 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-26 06:40:04,049 INFO Creating 1 session(s) with parameters:
2024-03-26 06:40:04,052 - skaha-client-skaha.session - INFO - {'name': '587731512085708963', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587731512085708963 8086-9101 /arc/projects/salvage/ALMA_data/587731512085708963/2017.1.01093.S/science_goal.uid___A001_X1288_Xee7/group.uid___A001_X1288_Xee8/member.uid___A001_X1288_Xee9', 'env': {}}
2024-03-26 06:40:04,052 INFO {'name': '587731512085708963', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587731512085708963 8086-9101 /arc/projects/salvage/ALMA_data/587731512085708963/2017.1.01093.S/science_goal.ui

Sesion ID: u45j4vlu


2024-03-26 06:40:08,438 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-26 06:40:08,438 INFO Creating 1 session(s) with parameters:
2024-03-26 06:40:08,440 - skaha-client-skaha.session - INFO - {'name': '587731187798507929', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587731187798507929 CP_7968_12705 /arc/projects/salvage/ALMA_data/587731512085708963/2017.1.01093.S/science_goal.uid___A001_X1288_Xee7/group.uid___A001_X1288_Xee8/member.uid___A001_X1288_Xee9', 'env': {}}
2024-03-26 06:40:08,440 INFO {'name': '587731187798507929', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587731187798507929 CP_7968_12705 /arc/projects/salvage/ALMA_data/587731512085708963/2017.1.01093.S/science

Sesion ID: noti4sz5


2024-03-26 06:40:09,653 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-26 06:40:09,653 INFO Creating 1 session(s) with parameters:
2024-03-26 06:40:09,655 - skaha-client-skaha.session - INFO - {'name': '588848900431216934', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '588848900431216934 HOT2_EI_CII_G09.v2.42 /arc/projects/salvage/ALMA_data/587731512085708963/2017.1.01093.S/science_goal.uid___A001_X1288_Xee7/group.uid___A001_X1288_Xee8/member.uid___A001_X1288_Xee9', 'env': {}}
2024-03-26 06:40:09,655 INFO {'name': '588848900431216934', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '588848900431216934 HOT2_EI_CII_G09.v2.42 /arc/projects/salvage/ALMA_data/587731512085708963/2017.1

...

Sesion ID: tlxvhtx0


2024-03-26 06:40:17,936 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-26 06:40:17,936 INFO Creating 1 session(s) with parameters:
2024-03-26 06:40:17,938 - skaha-client-skaha.session - INFO - {'name': '588015509828075649', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '588015509828075649 8081-3702 /arc/projects/salvage/ALMA_data/587731512085708963/2017.1.01093.S/science_goal.uid___A001_X1288_Xee7/group.uid___A001_X1288_Xee8/member.uid___A001_X1288_Xee9', 'env': {}}
2024-03-26 06:40:17,938 INFO {'name': '588015509828075649', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '588015509828075649 8081-3702 /arc/projects/salvage/ALMA_data/587731512085708963/2017.1.01093.S/science_goal.ui

Sesion ID: z8gwm1wa


2024-03-26 06:40:19,958 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-26 06:40:19,958 INFO Creating 1 session(s) with parameters:
2024-03-26 06:40:19,959 - skaha-client-skaha.session - INFO - {'name': '588015509826830376', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '588015509826830376 9194-3702 /arc/projects/salvage/ALMA_data/587731512085708963/2017.1.01093.S/science_goal.uid___A001_X1288_Xee7/group.uid___A001_X1288_Xee8/member.uid___A001_X1288_Xee9', 'env': {}}
2024-03-26 06:40:19,959 INFO {'name': '588015509826830376', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '588015509826830376 9194-3702 /arc/projects/salvage/ALMA_data/587731512085708963/2017.1.01093.S/science_goal.ui

Sesion ID: gz63nh2o
Sesion ID: stl6sobp


In [261]:
for i,sid in enumerate(session_ids):

    info = session.info(sid)[0]
    print(info['id'], info['status'], info['name'])

u45j4vlu Succeeded 587731512085708963-1
noti4sz5 Succeeded 587731511541432411-1
u2ziplvg Succeeded 587731187798507929-1
gyvwmrhx Running 588848900431216934-1
hz0ukdwh Running 588848900431216934-1
jern8xvf Succeeded 588015509826830376-1
tlxvhtx0 Failed 588015509828075649-1
z8gwm1wa Succeeded 588015509828075649-1
gz63nh2o Failed 588015509828075649-1
stl6sobp Succeeded 588015509826830376-1


In [260]:
for sid in session_ids:

    print(session.logs(sid)[sid])

## Identify Appropriate CASA Version and Run ScriptForPI.py

Restoring calibrations with scriptForPI.py requires the specific CASA version for which the script was designed. Finding the correct version of CASA to restore the calibration can be challenging, since the location where that information was stored has changed over the years. The following function is adapted from code provided to me by Helen Kirk. 

**KNOWN ISSUE**: ScriptForPI.py can take a while to run and is given a lower priority than interactive session on the CANFAR science platform. It is therefore entirely possible (likely even) that some jobs will get killed to make resources available for interactive sessions and then restarted at a later time when more resources become available. In the case of the download and unzip jobs, the code I have is pretty good at picking up where it left off. Is scriptForPI.py smart enough to realize what might have been corrupted and what may not have?

Possible solution: use a separate directory for intermediate scriptForPI.py files. 

Possible solution: delete all possibly relevant ScriptForPI.py products at the start of the bash script.

**KNOWN ISSUE**: Some casa pipeline versions are not available on the CANFAR science platform. I have it set up to default to the closest available version.

In [229]:
def get_casa_version(ID, PROJ, UID):

    version = 'unknown'
    calib = 'manual'

    # walk through the directory structure for this SDSS object observed for this ALMA Project 
    for root, dirs, files in os.walk(f"/arc/projects/salvage/ALMA_data/{ID}/{PROJ}/"):

        # if xml files are found, consider this object pipeline calibrated
        if len(glob.glob(f'{root}/*xml')) > 0:
            calib='pipeline'
            #break

        # save the correct file path for the member.uid directory (hard to predict otherwise)
        if "member.uid___"+UID in dirs:
            PATH = os.path.join(root, "member.uid___"+UID)        

    # if a pipeline_manifest.xml file exists, we can pull the appropriate casa version from it
    if len(glob.glob(f'{PATH}/script/member.uid___{UID}.*image.pipeline_manifest.xml'))>0:

        # if at least one pipeline_manifest.xml exists, it shouldn't matter which one you use, so just take the first
        file_to_check = glob.glob(f'{PATH}/script/member.uid___{UID}.*image.pipeline_manifest.xml')[0]

        # read in the lines of the xml file ...
        with open(file_to_check, 'r') as file:
            
            content = file.readlines()

        # search through the lines for the casa version name
        for line in content:

            # check if current line has < > formatting ...
            if len(line.split('<')) < 2:
                continue

            split_line = line.split('<')[1].split('=')

            # if there was a '=', it may be indicating the casa version
            if len(split_line) > 1:

                # if before the '=' was 'casaversion name'
                if split_line[0] == 'casaversion name':

                    # then save what comes after as the casa version
                    version = line.split('<')[1].split('=')[1].split('\"')[1]
                    method = '.xml files'


    # if there are no .xml files, try using the casa logs
    elif len(glob.glob(f'{PATH}/log/casa*.log'))>0:

        file_to_check = glob.glob(f'{PATH}/log/casa*.log')[0]

        # read in the lines of the log file ...
        with open(file_to_check, 'r') as file:
            
            content = file.readlines()

        # search through the lines for the casa version name
        for line in content:

            split_line = line.split('CASA Version')

            # if the current line was split by the above text, it must be present
            if len(split_line)>1:

                # extract casa version from line
                version = split_line[1].split(' ')[1].split('-')[0]
                version = convert_casa_version_name(version)
                method = 'CASA logs'

    # if there are no .xml files or casa log files, try searching in scriptForImaging.py for their casa version check
    elif len(glob.glob(f'{PATH}/script/*scriptForImaging.py'))>0:

        file_to_check = glob.glob(f'{PATH}/script/*scriptForImaging.py')[0]

        # read in the lines of the log file ...
        with open(file_to_check, 'r') as file:
            
            content = file.readlines()

        # search through the lines for the casa version name
        for line in content:

            split_line = line.split('ERROR: PLEASE USE THE SAME VERSION OF CASA THAT YOU USED FOR GENERATING THE SCRIPT: ')

            # if the current line was split by the above text, it must be present
            if len(split_line)>1:

                    # extract casa version from line
                    version = split_line[1].split("\'")[0]
                    method = 'scriptForImaging.py'
        
    else:

        print('No /script/*.pipeline_manifest.xml files, /log/casapy*.log files, or /script/*scriptForImaging.py.')
        method = 'failed'
        
    return version, calib, method


def convert_casa_version_name(version_in):

    # translation here: https://almascience.nrao.edu/processing/science-pipeline#version
    
    if version_in == '5.6.42831':
        version_out='5.6.1'
    elif version_in == '5.4.42866':
        version_out='5.4.0'
    elif version_in == '5.1.40896':
        version_out='5.1.1'
    elif version_in == '4.7.39732':
        version_out='4.7.2'
    elif version_in == '4.7.38335':
        version_out='4.7.0'
    elif version_in == '4.5.36091':
        version_out='4.5.2'
    elif version_in == '4.5.35996':
        version_out='4.5.1'
    elif version_in == '4.3.32491':
        version_out='4.3.1'
    elif version_in == '4.2.30986':
        version_out='4.2.2'
    else:
        print('Conversion not supported, returning the same version.')
        version_out=version_in

    return version_out

def casa_version_to_canfar_image(version):

    # translation here: https://almascience.nrao.edu/processing/science-pipeline#version
    
    if version == '5.6.1-8':
        image='images.canfar.net/casa-5/casa:5.6.1-8-pipeline'

    elif version == '5.4.0-70':
        print('NO HEADLESS IMAGE FOR CASA 5.4.0-70 PIPELINE, reverting to 5.6.1-8 because its the closest version.')
        image='images.canfar.net/casa-5/casa:5.6.1-8-pipeline'
        
    elif version == '5.4.0-68':
        print('NO HEADLESS IMAGE FOR CASA 5.4.0-68 PIPELINE, reverting to 5.6.1-8 because its the closest version.')
        image='images.canfar.net/casa-5/casa:5.6.1-8-pipeline'
        
    elif version == '5.1.1-5':
        print('NO HEADLESS IMAGE FOR CASA 5.1.1-5 PIPELINE, reverting to 5.6.1-8 because its the closest version.')
        image='images.canfar.net/casa-5/casa:5.6.1-8-pipeline'

    elif version == '5.1.1':
        print('NO HEADLESS IMAGE FOR CASA 5.1.1 PIPELINE, reverting to 5.6.1-8 because its the closest version.')
        image='images.canfar.net/casa-5/casa:5.6.1-8-pipeline'
        
    elif version == '4.7.2':
        print('NO HEADLESS IMAGE FOR CASA 4.7.2 PIPELINE, reverting to 4.3.1 because its the closest version.')
        image='images.canfar.net/casa-4/casa:4.3.1-pipe'
        
    elif version == '4.7.0':
        print('NO HEADLESS IMAGE FOR CASA 4.7.0 PIPELINE, reverting to 4.3.1 because its the closest version.')
        image='images.canfar.net/casa-4/casa:4.3.1-pipe'
        
    elif version == '4.5.2':
        print('NO HEADLESS IMAGE FOR CASA 4.5.2 PIPELINE, reverting to 4.3.1 because its the closest version.')
        image='images.canfar.net/casa-4/casa:4.3.1-pipe'
        
    elif version == '4.5.1':
        print('NO HEADLESS IMAGE FOR CASA 4.5.1 PIPELINE, reverting to 4.3.1 because its the closest version.')
        image='images.canfar.net/casa-4/casa:4.3.1-pipe'
        
    elif version == '4.3.1':
        image='images.canfar.net/casa-4/casa:4.3.1-pipe'
        
    elif version == '4.2.2':
        image='images.canfar.net/casa-4/casa:4.2.2-pipe'
        
    else:
        print(f'Your CASA version ({version}) is not supported, defaulting to most recent: 6.5.4-9.')
        image = 'images.canfar.net/casa-6/casa:6.5.4-9-pipeline'

    return image
        

# test the function

for i in range(len(objID_sample)):

    ID = objID_sample[i]
    MUID = muid_sample[i]
    PROJ = proj_sample[i]

    UID =  MUID.split('/')[2] + '_' + MUID.split('/')[3] + '_' + MUID.split('/')[4]

    version, calib, method = get_casa_version(ID, PROJ, UID)
    print(ID, version, calib, method)
    print(casa_version_to_canfar_image(version))
    print()

587724198283247682 5.6.1-8 pipeline .xml files
images.canfar.net/casa-5/casa:5.6.1-8-pipeline

587730848501530706 5.6.1-8 pipeline .xml files
images.canfar.net/casa-5/casa:5.6.1-8-pipeline

587730847423791750 5.6.1-8 pipeline .xml files
images.canfar.net/casa-5/casa:5.6.1-8-pipeline

587731173842027508 5.4.0-70 pipeline .xml files
NO HEADLESS IMAGE FOR CASA 5.4.0-70 PIPELINE, reverting to 5.6.1-8 because its the closest version.
images.canfar.net/casa-5/casa:5.6.1-8-pipeline

587730847427265072 5.1.1-5 pipeline .xml files
NO HEADLESS IMAGE FOR CASA 5.1.1-5 PIPELINE, reverting to 5.6.1-8 because its the closest version.
images.canfar.net/casa-5/casa:5.6.1-8-pipeline

587731173842027508 5.4.0-68 pipeline .xml files
NO HEADLESS IMAGE FOR CASA 5.4.0-68 PIPELINE, reverting to 5.6.1-8 because its the closest version.
images.canfar.net/casa-5/casa:5.6.1-8-pipeline

587731512085708963 5.6.1-8 pipeline .xml files
images.canfar.net/casa-5/casa:5.6.1-8-pipeline

588015508213989555 5.4.0-70 pipeli

In [232]:
from skaha.images import Images

images = Images()
images.fetch(kind = 'headless')

['images.canfar.net/canfar/base-3.12:v0.4.1',
 'images.canfar.net/canucs/test:1.2.5',
 'images.canfar.net/casa-4/casa:4.2.2-pipe',
 'images.canfar.net/casa-4/casa:4.3.1-pipe',
 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline',
 'images.canfar.net/casa-5/casa:5.6.3-19-pipeline',
 'images.canfar.net/casa-5/casa:5.6.2-6-pipeline',
 'images.canfar.net/casa-6/casa:6.5.6-22',
 'images.canfar.net/casa-6/casa:6.6.0-20',
 'images.canfar.net/casa-6/casa:6.6.3-22',
 'images.canfar.net/casa-6/casa:6.1.1-15-pipeline',
 'images.canfar.net/casa-6/casa:6.5.4-9-pipeline',
 'images.canfar.net/casa-6/casa:6.2.1-7-pipeline',
 'images.canfar.net/casa-6/casa:6.5.3-28-pipeline',
 'images.canfar.net/casa-6/casa:6.4.1-12-pipeline',
 'images.canfar.net/cirada/possumpipelineprefect-3.11:v0.9.0',
 'images.canfar.net/cirada/possumpipelineprefect-3.11:latest',
 'images.canfar.net/cirada/possumpipelineprefect-3.11:v0.8.0',
 'images.canfar.net/cirada/possumpipelineprefect-3.11:v0.7.0',
 'images.canfar.net/cirada/poss

In [252]:
# select appropriate resources
cmd = '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh'
ram=6 # testing with slightly less RAM
cores=2

session_ids = []

#rerun_targets = ['587731513153159199', '587727220865040712']
rerun_targets = ['587731512085708963']

# loop over galaxies and launch jobs
for i in range(len(objID_sample)):

    ID = objID_sample[i]
    MUID = muid_sample[i]
    PROJ = proj_sample[i]

    UID =  MUID.split('/')[2] + '_' + MUID.split('/')[3] + '_' + MUID.split('/')[4]

    # identify and select the appropriate CASA version
    version, calib, method = get_casa_version(ID, PROJ, UID)
    image = casa_version_to_canfar_image(version)
    print(ID, version, calib, method, image)

    # running only on problem galaxies
    #if ID not in rerun_targets:
    #    continue

    # search all directories in the ALMA project folder for the relevant data
    for root, dirs, files in os.walk(f"/arc/projects/salvage/ALMA_data/{ID}/{PROJ}/"):
        if "member.uid___"+UID in dirs:
            PATH = os.path.join(root, "member.uid___"+UID)

    # if the data has already been calibrated, choose if it will be skipped
    skip_completed = True
    ms_filepath = PATH + '/calibrated/'
    ms_files = glob.glob(ms_filepath + '*.ms*')
    if (len(ms_files)>0) & skip_completed:
        continue

    # launch headless session on the science platform
    session = Session()
    session_id = session.create(
        name  = ID,
        image = image,
        cores = cores,
        ram   = ram,
        kind  = "headless",
        cmd   = cmd,
        args  = f'{ID} {PROJ} {PATH}'
    )

    print("Sesion ID: {}".format(session_id[0]))
    session_ids.append(session_id[0])


2024-03-26 05:55:43,841 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-26 05:55:43,841 INFO Creating 1 session(s) with parameters:
2024-03-26 05:55:43,843 - skaha-client-skaha.session - INFO - {'name': '587724198283247682', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 6, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587724198283247682 2019.1.00260.S /arc/projects/salvage/ALMA_data/587724198283247682/2019.1.00260.S/science_goal.uid___A001_X1465_X3527/group.uid___A001_X1465_X3528/member.uid___A001_X1465_X3529', 'env': {}}
2024-03-26 05:55:43,843 INFO {'name': '587724198283247682', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 6, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587724198283247682 2019.1.00260.S /arc/projects/salvage/ALMA_data/587724198283247682

587724198283247682 5.6.1-8 pipeline .xml files images.canfar.net/casa-5/casa:5.6.1-8-pipeline
Sesion ID: vc25sk2i
587730848501530706 5.6.1-8 pipeline .xml files images.canfar.net/casa-5/casa:5.6.1-8-pipeline


2024-03-26 05:55:48,840 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-26 05:55:48,840 INFO Creating 1 session(s) with parameters:
2024-03-26 05:55:48,841 - skaha-client-skaha.session - INFO - {'name': '587730847423791750', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 6, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587730847423791750 2019.1.00260.S /arc/projects/salvage/ALMA_data/587730847423791750/2019.1.00260.S/science_goal.uid___A001_X1465_X3513/group.uid___A001_X1465_X3514/member.uid___A001_X1465_X3515', 'env': {}}
2024-03-26 05:55:48,841 INFO {'name': '587730847423791750', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 6, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587730847423791750 2019.1.00260.S /arc/projects/salvage/ALMA_data/587730847423791750

587730847423791750 5.6.1-8 pipeline .xml files images.canfar.net/casa-5/casa:5.6.1-8-pipeline
Sesion ID: ub2g815e


2024-03-26 05:55:51,928 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-26 05:55:51,928 INFO Creating 1 session(s) with parameters:
2024-03-26 05:55:51,930 - skaha-client-skaha.session - INFO - {'name': '587731173842027508', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 6, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587731173842027508 2018.1.00541.S /arc/projects/salvage/ALMA_data/587731173842027508/2018.1.00541.S/science_goal.uid___A001_X133d_Xea1/group.uid___A001_X133d_Xea2/member.uid___A001_X133d_Xea3', 'env': {}}
2024-03-26 05:55:51,930 INFO {'name': '587731173842027508', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 6, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587731173842027508 2018.1.00541.S /arc/projects/salvage/ALMA_data/587731173842027508/20

...

NO HEADLESS IMAGE FOR CASA 5.4.0-70 PIPELINE, reverting to 5.6.1-8 because its the closest version.
587742060530696249 5.4.0-70 pipeline .xml files images.canfar.net/casa-5/casa:5.6.1-8-pipeline
Sesion ID: mrjxcs5u
NO HEADLESS IMAGE FOR CASA 5.1.1-5 PIPELINE, reverting to 5.6.1-8 because its the closest version.
587731186724569574 5.1.1-5 pipeline .xml files images.canfar.net/casa-5/casa:5.6.1-8-pipeline


2024-03-26 05:57:47,976 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-26 05:57:47,976 INFO Creating 1 session(s) with parameters:
2024-03-26 05:57:48,007 - skaha-client-skaha.session - INFO - {'name': '587731186187632690', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 6, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587731186187632690 2018.1.00541.S /arc/projects/salvage/ALMA_data/587731186187632690/2018.1.00541.S/science_goal.uid___A001_X133d_Xe65/group.uid___A001_X133d_Xe66/member.uid___A001_X133d_Xe67', 'env': {}}
2024-03-26 05:57:48,007 INFO {'name': '587731186187632690', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 6, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587731186187632690 2018.1.00541.S /arc/projects/salvage/ALMA_data/587731186187632690/20

NO HEADLESS IMAGE FOR CASA 5.4.0-70 PIPELINE, reverting to 5.6.1-8 because its the closest version.
587731186187632690 5.4.0-70 pipeline .xml files images.canfar.net/casa-5/casa:5.6.1-8-pipeline
Sesion ID: krtuoycr
NO HEADLESS IMAGE FOR CASA 5.1.1-5 PIPELINE, reverting to 5.6.1-8 because its the closest version.
587724240159834264 5.1.1-5 pipeline .xml files images.canfar.net/casa-5/casa:5.6.1-8-pipeline


2024-03-26 05:57:55,034 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-26 05:57:55,034 INFO Creating 1 session(s) with parameters:
2024-03-26 05:57:55,035 - skaha-client-skaha.session - INFO - {'name': '587724234256351256', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 6, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587724234256351256 2019.1.00260.S /arc/projects/salvage/ALMA_data/587724234256351256/2019.1.00260.S/science_goal.uid___A001_X1465_X351b/group.uid___A001_X1465_X351c/member.uid___A001_X1465_X351d', 'env': {}}
2024-03-26 05:57:55,035 INFO {'name': '587724234256351256', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 6, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587724234256351256 2019.1.00260.S /arc/projects/salvage/ALMA_data/587724234256351256

587724234256351256 5.6.1-8 pipeline .xml files images.canfar.net/casa-5/casa:5.6.1-8-pipeline
Sesion ID: d4ej2es1
NO HEADLESS IMAGE FOR CASA 5.1.1-5 PIPELINE, reverting to 5.6.1-8 because its the closest version.
587739719757922345 5.1.1-5 pipeline .xml files images.canfar.net/casa-5/casa:5.6.1-8-pipeline


In [253]:
for i,sid in enumerate(session_ids):

    try:
        info = session.info(sid)[0]
        print(info['name'], info['id'], info['status'])

    except:

        print(session.info(sid))

587724198283247682-1 vc25sk2i Running
587730847423791750-1 ub2g815e Running
587731173842027508-1 h9ymdxo7 Running
588015508213989555-1 cf6lsqzy Succeeded
587731185650827595-1 la09df7a Succeeded
587727220865040712-1 ggwqh3a6 Running
587731511545626673-1 p94a72ni Succeeded
587731187797393472-1 v2f8ws6f Succeeded
587731511545233578-1 v01tpqos Succeeded
587731512085708963-1 sqk2eh13 Succeeded
587731511541432411-1 kwk2t267 Succeeded
587731187798507929-1 s1xx199e Succeeded
588848900431216934-1 p1eb3mkw Succeeded
588015509826830376-1 zh1gwyf3 Succeeded
588015509828075649-1 b3430094 Succeeded
588015509828075649-1 mca12ck7 Running
588015509828075649-1 ix2vpume Running
588015509826830376-1 yp4ey06h Succeeded
587724197746311240-1 rsv6lyxi Succeeded
587741386202218596-1 fxu0zcpz Succeeded
588015508731723912-1 r1b6cqgp Succeeded
587731513694355564-1 w4yefylz Succeeded
587731513153159199-1 sekbn7d7 Succeeded
587730772799914185-1 vq1eux1b Succeeded
587731512619696292-1 t4f74l5n Succeeded
587741490891

In [61]:
for sid in session_ids:
    
    print(session.logs(sid)[sid])

exec /arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh: exec format error



## Make Measurement Set Key

In [109]:
key_dir = '/arc/projects/salvage/phangs_imaging_scripts/phangs-alma_keys/'

ms_key_out = 'ms_file_key_salvage.txt'
ms_key_tmp = 'ms_file_key_template.txt'

# prep by copy current ms key to a template file with "__DATA__" where data should go (DONE)

# get template table (formatting with no file paths)
out = open(key_dir + ms_key_tmp, 'r')
out_data = out.read()
out.close()

# wipe old content from output file
out = open(key_dir + ms_key_out, 'w')
out.close()

# initialize objID input tracker
objID_in_file = []

# initialize string
out_str = ''

for i in range(len(objID_sample)):

    ID = objID_sample[i]
    MUID = muid_sample[i]
    PROJ = proj_sample[i]

    UID =  MUID.split('/')[2] + '_' + MUID.split('/')[3] + '_' + MUID.split('/')[4]

    # search all directories in the ALMA project folder for the relevant data
    for root, dirs, files in os.walk(f"/arc/projects/salvage/ALMA_data/{ID}/{PROJ}/"):
        if "member.uid___"+UID in dirs:
            PATH = os.path.join(root, "member.uid___"+UID)

    # prep file path for PHANGS-ALMA pipeline
    ms_root = '/arc/projects/salvage/ALMA_data/'
    ms_filepath = PATH.replace(ms_root, '') # wipe ms_root directory so it can be added separately
    ms_filepath += '/calibrated/' # add calibrated on the end so that it points to the calibrated data

    # until I discover more nuance, take all measurement set files from this directory and let PHANGS deal with them
    ms_files = glob.glob(ms_root + ms_filepath + '*.ms*')

    print(ms_root + ms_filepath, os.path.isdir(ms_root + ms_filepath), len(ms_files))

    if len(ms_files)<1:
        #checking if files are missing because of bug in previous stages or this one...
        print('No MS files found in ', ms_filepath)

    # check if there are visibilities for this target from another project and add on top if there is...
    init = len(np.array(objID_in_file)[np.array(objID_in_file)==ID])

    # add each measurement set to ms key
    for j in range(init, init + len(ms_files)):

        ms_path = ms_files[j-init].replace(ms_root, '') # wipe ms_root directory so it can be added separately
        obs_num = j+1 # record different measurement sets as different observations
    
        out_str += f"{ID} {ID}_{obs_num} all 12m {obs_num} {ms_path}\n" 
        # need to accomodate other observations of the same objID?

        objID_in_file.append(ID)

# replace placeholder "__DATA__" in template with formatted data
out_data = out_data.replace('__DATA__', out_str)
    
# write to new key file
out = open(key_dir + ms_key_out, 'w')
out.write(out_data)
out.close()

/arc/projects/salvage/ALMA_data/587724198283247682/2019.1.00260.S/science_goal.uid___A001_X1465_X3527/group.uid___A001_X1465_X3528/member.uid___A001_X1465_X3529/calibrated/ True 2
/arc/projects/salvage/ALMA_data/587730848501530706/2019.1.00260.S/science_goal.uid___A001_X1465_X34e3/group.uid___A001_X1465_X34e4/member.uid___A001_X1465_X34e5/calibrated/ True 1
/arc/projects/salvage/ALMA_data/587730847423791750/2019.1.00260.S/science_goal.uid___A001_X1465_X3513/group.uid___A001_X1465_X3514/member.uid___A001_X1465_X3515/calibrated/ True 1
/arc/projects/salvage/ALMA_data/587731173842027508/2018.1.00541.S/science_goal.uid___A001_X133d_Xea1/group.uid___A001_X133d_Xea2/member.uid___A001_X133d_Xea3/calibrated/ True 2
/arc/projects/salvage/ALMA_data/587730847427265072/2017.1.01093.S/science_goal.uid___A001_X1288_Xeef/group.uid___A001_X1288_Xef0/member.uid___A001_X1288_Xef1/calibrated/ True 1
/arc/projects/salvage/ALMA_data/587731173842027508/2018.1.00558.S/science_goal.uid___A001_X133d_Xf47/group

## Make distance key

In [110]:
key_dir = '/arc/projects/salvage/phangs_imaging_scripts/phangs-alma_keys/'

dist_key_out = 'distance_key_salvage.txt'
dist_key_tmp = 'distance_key_template.txt'

# prep by copy current distance key to a template file with "__DATA__" where data should go (DONE)

# get template table (formatting with no coordinates)
out = open(key_dir + dist_key_tmp, 'r')
out_data = out.read()
out.close()

# initialize string
out_str = ''

for i in range(len(objID_sample)):

    ID   = objID_sample[i]
    NAME = name_sample[i]
    Z    = z_sample[i]

    DIST = cosmo.angular_diameter_distance(Z).value

    out_str += f"{ID},{DIST}\n" 
    # need to accomodate other observations of the same objID?

# replace __DATA__ in template with formatted data
out_data = out_data.replace('__DATA__', out_str)

# write to new key file
out = open(key_dir + dist_key_out, 'w')
out.write(out_data)
out.close()

## Make target definitions key

In [99]:
key_dir = '/arc/projects/salvage/phangs_imaging_scripts/phangs-alma_keys/'

targ_key_out = 'target_definitions_salvage.txt'
targ_key_tmp = 'target_definitions_template.txt'

# prep by copy current target definitions key to a template file with "__DATA__" where data should go (DONE)

# prep for sys vel calculation
restfreq = 115.27120 * u.GHz  # rest frequency of 12 CO 1-0 in GHz
freq_to_vel = u.doppler_radio(restfreq)

# get template table (formatting with no coordinates)
out = open(key_dir + targ_key_tmp, 'r')
out_data = out.read()
out.close()

# initialize string
out_str = ''

# format the ra and dec's for html table
for i in range(len(objID_sample)):

    ID   = objID_sample[i]
    NAME = name_sample[i]
    Z    = z_sample[i]
    RA   = ra_sample[i]
    DEC  = dec_sample[i]

    c = SkyCoord(ra= RA*u.degree, dec= DEC*u.degree, frame='icrs')
    coord_str = c.to_string('hmsdms')

    sys_vel = (restfreq/(1+Z)).to(u.km / u.s, equivalencies=freq_to_vel).value
    velwidth = 1000

    out_str += f"{ID} {coord_str} {sys_vel} {velwidth}\n" 
    # need to accomodate other observations of the same objID?

# replace __DATA__ in template with formatted data
out_data = out_data.replace('__DATA__', out_str)

# write to new key file
out = open(key_dir + targ_key_out, 'w')
out.write(out_data)
out.close()

## Run PHANGS Pipeline

In [111]:
# select appropriate resources

image = "images.canfar.net/casa-6/casa:6.5.6-22"
cmd = '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_pipeline.sh'
ram=8
cores=2

session_ids = []

#rerun_targets = ['587731513153159199']
#rerun_targets = ['587731511545233578']
rerun_targets = ['587731512085708963']

# loop over galaxies and launch jobs
for i in range(len(objID_sample)):

    ID = objID_sample[i]
    NAME = name_sample[i]

    # running only on problem galaxies
    if ID not in rerun_targets:
        continue
    
    # launch headless session on the science platform
    session = Session()
    session_id = session.create(
        name  = ID,
        image = image,
        cores = cores,
        ram   = ram,
        kind  = "headless",
        cmd   = cmd,
        args  = f'{NAME} {ID}'
    )

    print("Sesion ID: {}".format(session_id[0]))
    session_ids.append(session_id[0])


2024-03-22 06:22:15,820 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-22 06:22:15,820 INFO Creating 1 session(s) with parameters:
2024-03-22 06:22:15,823 - skaha-client-skaha.session - INFO - {'name': '587731512085708963', 'image': 'images.canfar.net/casa-6/casa:6.5.6-22', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_pipeline.sh', 'args': '8086-3704 587731512085708963', 'env': {}}
2024-03-22 06:22:15,823 INFO {'name': '587731512085708963', 'image': 'images.canfar.net/casa-6/casa:6.5.6-22', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_pipeline.sh', 'args': '8086-3704 587731512085708963', 'env': {}}
2024-03-22 06:22:17,625 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-22 06:22:17,625 INFO Creating 1 session(s) with parameters:
2024-03-22 06:22:17,628 - skaha-client-skaha.session - 

Sesion ID: loqg3g96
Sesion ID: j2f2bkjs


In [113]:
for i,sid in enumerate(session_ids):

    info = session.info(sid)[0]
    print(info['id'], info['status'], info['name'])

loqg3g96 Pending 587731512085708963-1
j2f2bkjs Pending 587731512085708963-1


In [83]:
for sid in session_ids:
    
    print(session.logs(sid)[sid])

/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_pipeline.sh: line 35: syntax error: unexpected end of file



## Run PHANGS moments

In [116]:
# select appropriate resources

image = "images.canfar.net/skaha/astroml:latest"
#image = "images.canfar.net/casa-5/casa:5.6.1-8-pipeline"
cmd = '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh'
ram=4
cores=2

session_ids = []

# testing targets that have empty mom0 maps
#rerun_targets = ['587726033309335775', '588010359075700997', '587727944034091239']
rerun_targets = ['587731513153159199']

# loop over galaxies and launch jobs
for i in range(len(objID_sample)):

    ID = objID_sample[i]
    NAME = name_sample[i]

    # running only on problem galaxies
    if ID not in rerun_targets:
        continue

    # launch headless session on the science platform
    session = Session()
    session_id = session.create(
        name  = ID,
        image = image,
        cores = cores,
        ram   = ram,
        kind  = "headless",
        cmd   = cmd,
        args  = f'{NAME} {ID}'
    )

    print("Sesion ID: {}".format(session_id[0]))
    session_ids.append(session_id[0])


2024-03-22 07:54:35,226 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-22 07:54:35,226 INFO Creating 1 session(s) with parameters:
2024-03-22 07:54:35,228 - skaha-client-skaha.session - INFO - {'name': '587731513153159199', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': 'CP_8078_6104 587731513153159199', 'env': {}}
2024-03-22 07:54:35,228 INFO {'name': '587731513153159199', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': 'CP_8078_6104 587731513153159199', 'env': {}}


Sesion ID: nigzzwqj


In [118]:
for i,sid in enumerate(session_ids):

    info = session.info(sid)[0]
    print(info['id'], info['status'])

nigzzwqj Succeeded


In [152]:
for sid in session_ids:
    
    print(session.logs(sid)[sid])

Execute bash script.
NAME = 587731173842027508_8623-6104
OBJID = 587731173842027508
Downgrading astropy
Defaulting to user installation because normal site-packages is not writeable
  Using cached astropy-5.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (9.5 kB)
Using cached astropy-5.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (10.1 MB)
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/arc/home/swilkinson/.local/lib/python3.10/site-packages/astropy/io/votable/data/ucd1p-words.txt'


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Running derived pipeline...
[DEBU] [                 <module>] casa_enabled = False
[INFO] [        build_key_handler] 
[INFO] [        build_key_handler] &%&%&%&%&%&%&%&%&%&%&%&%&%&%&&%&%&%&%&%&%&%&%&%&%
[INFO] [        build_key_handler] I